<a href="https://colab.research.google.com/github/InowaR/Big-Data/blob/main/lesson2/homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install opendatasets

In [3]:
import opendatasets as od
import pandas as pd
import time
from pathlib import Path

In [5]:
dataset_path = Path('new-york-city-airbnb-open-data', 'AB_NYC_2019.csv')
if not dataset_path.is_file():
    od.download('https://www.kaggle.com/datasets/dgomonov/new-york-city-airbnb-open-data')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: inowar11
Your Kaggle Key: ··········


100%|██████████| 2.44M/2.44M [00:00<00:00, 137MB/s]

In [6]:
! pip install pyspark
! pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=cc266be0b960559b263b8900a11d36b28ed67a721210c2c50fc8cc4a6b274cbd
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [188]:
from pyspark.sql import SparkSession
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1" # без этой строчки у нас будет возникать постоянное предупреждение с просьбой установить эту переменную в значение 1, что мы заранее и делаем


spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark

In [189]:
df = spark.read.csv('new-york-city-airbnb-open-data/AB_NYC_2019.csv', header=True, inferSchema=True)

df.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- calculated_host_listings_count: string (nullable = true)
 |-- availability_365: integer (nullable = true)



In [203]:
df.show(5)

+----+--------------------+-------+-----------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|  id|                name|host_id|  host_name|neighbourhood_group|neighbourhood|latitude|longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|
+----+--------------------+-------+-----------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|2539|Clean & quiet apt...|   2787|       John|           Brooklyn|   Kensington|40.64749|-73.97237|   Private room|  149|             1|                9| 2018-10-19|             0.21|                             6|             365|
|2595|Skylit Midtown Ca...|   2845|   Jennifer|          Manhatt

In [525]:
df.count()

49079

In [496]:
df_price = df.select("price").withColumn("price", df.price.cast("double"))

avg_price = df_price.agg({"price": "avg"}).collect()[0]["avg(price)"]
var_price = df_price.agg({"price": "var_samp"}).collect()[0]["var_samp(price)"]

print(f"Среднее значение price: {avg_price}")
print(f"Дисперсия price: {var_price}")


Среднее значение price: 152.22296299343384
Дисперсия price: 56902.04073527261


In [551]:
def mapper(input_path):
  a = []
  with open(input_path, "r") as f:
    for line in f:
        fields = line.strip().split(",")
        try:
          n = float(fields[9])
          if n >= 0:
            a.append(n)
        except:
          pass
  return a


input_path = "new-york-city-airbnb-open-data/AB_NYC_2019.csv"

a = mapper(input_path)
print(len(a))

42116


In [574]:
def reducer(first, second):
  global n
  global mean
  global M2
  global delta

  n += 1
  delta = second - mean
  mean += delta / n
  M2 += (delta)**2 / n

  return n, mean, M2

In [576]:
from functools import reduce
n, mean, M2 = reduce(reducer, mapper(input_path))
print(n, mean, M2)
n = 0
mean = 0
M2 = 0
delta = 0

42115 152.5571814500772 358385.5009220167
